In [1]:
import pandas as pd
import pandasql as ps

In [2]:
pathPedido = '../data/pedido.csv'
pathItemPedido = '../data/item_pedido.csv'

In [3]:
pedido = pd.read_csv(pathPedido)
itemPedido = pd.read_csv(pathItemPedido)

In [4]:
pathproducts = '../data_curso/olist_products_dataset.csv'
pathProduto = '../data/produto.csv'

In [5]:
produto = pd.read_csv(pathproducts)

In [6]:
produto.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [7]:
produto = produto.rename(columns = {"product_id" : "idProduto", 
                                    "product_category_name" : "descCategoria",
                                    "product_name_lenght" : "nrTamanhoNome", 
                                    "product_description_lenght" : "nrTamanhoDescricao",
                                    "product_photos_qty" : "nrFotos",
                                    "product_weight_g" : "vlPesoGramas",
                                    "product_length_cm" : "vlComprimentoCm",
                                    "product_height_cm" : "vlAlturaCm",
                                    "product_width_cm" : "vlLarguraCm"
    
})

In [8]:
produto.head()

,idProduto,descCategoria,nrTamanhoNome,nrTamanhoDescricao,nrFotos,vlPesoGramas,vlComprimentoCm,vlAlturaCm,vlLarguraCm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [9]:
#produto.to_csv(pathProduto, index = False, encoding = 'utf-8')

In [10]:
tb_join = ps.sqldf("""
                        select distinct
                        t2.idVendedor,
                        t3.*
                        
                        from pedido as t1
                        
                        left join itemPedido as t2
                        on t1.idPedido = t2.idPedido
                        
                        left join produto as t3
                        on t2.idProduto = t3.idProduto
                        
                         WHERE t1.dtPedido < '2018-01-01'
                         and t1.dtPedido >= '2017-07-01'
                         and t2.idVendedor is not null
                """)

In [11]:
tb_join.head()

,idVendedor,idProduto,descCategoria,nrTamanhoNome,nrTamanhoDescricao,nrFotos,vlPesoGramas,vlComprimentoCm,vlAlturaCm,vlLarguraCm
0,3504c0cb71d7fa48d967e0e4c94d59d9,87285b34884572647811a353c7ac498a,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0
1,66922902710d126a0e7d26b0e3805106,d0b61bfb1de832b15ba9d266ca96e5b0,pet_shop,59.0,468.0,3.0,450.0,30.0,10.0,20.0
2,8581055ce74af1daba164fdbd55a40de,060cb19345d90064d1015407193c233d,automotivo,49.0,608.0,1.0,7150.0,65.0,10.0,65.0
3,7c67e1448b00f6e969d365cea6b010ab,9a78fb9862b10749a117f7fc3c31f051,moveis_escritorio,45.0,527.0,1.0,9750.0,42.0,41.0,42.0
4,87142160b41353c4e5fca2360caf6f92,f7e0fa615b386bc9a8b9eb52bc1fff76,informatica_acessorios,59.0,2574.0,1.0,325.0,21.0,21.0,21.0


In [12]:
tb_summary = ps.sqldf("""
                        select idVendedor,
                        avg(coalesce(nrFotos, 0)) as avgFotos,
                        avg(coalesce(vlComprimentoCm * vlAlturaCm * vlLarguraCm, 0)) as avgVolumeProduto,
                        min(coalesce(vlComprimentoCm * vlAlturaCm * vlLarguraCm, 0)) as MinimoVolumeProduto,
                        max(coalesce(vlComprimentoCm * vlAlturaCm * vlLarguraCm, 0)) as MaximoVolumeProduto
                        
                        from tb_join
                        
                        group by idVendedor
                """)

In [13]:
tb_summary.head()

,idVendedor,avgFotos,avgVolumeProduto,MinimoVolumeProduto,MaximoVolumeProduto
0,0015a82c2db000af6aaaf3ae2ecb0532,2.000000,61920.000000,61920.0,61920.0
1,001cca7ae9ae17fb1caed9dfb1094831,1.750000,21437.250000,19656.0,24640.0
2,002100f778ceb8431b7a1020ff7ab48f,1.000000,8499.200000,4096.0,9600.0
3,003554e2dce176b5555353e4f3555ac8,0.000000,21000.000000,21000.0,21000.0
4,004c9cd9d87a3c30c522c48c4fc07416,0.911111,21858.511111,4176.0,136958.0


In [14]:
tb_categoria = ps.sqldf("""
                        select descCategoria
                        
                        from itemPedido as t2
                        
                        left join produto as t3
                        on t2.idProduto = t3.idProduto
                        
                        and t2.idVendedor is not null
                        
                        group by descCategoria
                        order by count(distinct idPedido) desc
                        limit 15
                """)

In [15]:
tb_categoria.head(20)

,descCategoria
0,cama_mesa_banho
1,beleza_saude
2,esporte_lazer
3,informatica_acessorios
4,moveis_decoracao
5,utilidades_domesticas
6,relogios_presentes
7,telefonia
8,automotivo
9,brinquedos


In [16]:
tb_categoria.shape

(15, 1)

In [17]:
tb_summary = ps.sqldf("""
                        select 
                        '2018-01-01' as dtReference,
                        idVendedor,
                        avg(coalesce(nrFotos, 0)) as avgFotos,
                        avg(coalesce(vlComprimentoCm * vlAlturaCm * vlLarguraCm, 0)) as avgVolumeProduto,
                        min(coalesce(vlComprimentoCm * vlAlturaCm * vlLarguraCm, 0)) as MinimoVolumeProduto,
                        max(coalesce(vlComprimentoCm * vlAlturaCm * vlLarguraCm, 0)) as MaximoVolumeProduto,
                        
                        cast(count (distinct case when descCategoria = 'cama_mesa_banho' then idProduto end) as double) / cast(count(distinct idProduto) as double) as pct_cama_mesa_banho,
                        cast(count (distinct case when descCategoria = 'beleza_saude' then idProduto end) as double) / cast(count(distinct idProduto) as double) as pct_beleza_saude,
                        cast(count (distinct case when descCategoria = 'esporte_lazer' then idProduto end) as double) / cast(count(distinct idProduto) as double) as pct_esporte_lazer,
                        cast(count (distinct case when descCategoria = 'informatica_acessorios' then idProduto end) as double) / cast(count(distinct idProduto) as double) as pct_informatica_acessorios,
                        cast(count (distinct case when descCategoria = 'moveis_decoracao' then idProduto end) as double) / cast(count(distinct idProduto) as double) as pct_moveis_decoracao,
                        cast(count (distinct case when descCategoria = 'utilidades_domesticas' then idProduto end) as double) / cast(count(distinct idProduto) as double) as pct_utilidades_domesticas,
                        cast(count (distinct case when descCategoria = 'relogios_presentes' then idProduto end) as double) / cast(count(distinct idProduto) as double) as pct_relogios_presentes,
                        cast(count (distinct case when descCategoria = 'telefonia' then idProduto end) as double) / cast(count(distinct idProduto) as double) as pct_telefonia,
                        cast(count (distinct case when descCategoria = 'automotivo' then idProduto end) as double) / cast(count(distinct idProduto) as double) as pct_automotivo,
                        cast(count (distinct case when descCategoria = 'brinquedos' then idProduto end) as double) / cast(count(distinct idProduto) as double) as pct_brinquedos,
                        cast(count (distinct case when descCategoria = 'cool_stuff' then idProduto end) as double) / cast(count(distinct idProduto) as double) as pct_cool_stuff,
                        cast(count (distinct case when descCategoria = 'ferramentas_jardim' then idProduto end) as double) / cast(count(distinct idProduto) as double) as pct_ferramentas_jardim,
                        cast(count (distinct case when descCategoria = 'perfumaria' then idProduto end) as double) / cast(count(distinct idProduto) as double) as pct_perfumaria,
                        cast(count (distinct case when descCategoria = 'bebes' then idProduto end) as double) / cast(count(distinct idProduto) as double) as pct_bebes,
                        cast(count (distinct case when descCategoria = 'eletronicos' then idProduto end) as double) / cast(count(distinct idProduto) as double) as pct_eletronicos
                        
                        from tb_join
                        
                        group by idVendedor
                """)

In [18]:
tb_summary.head()

,dtReference,idVendedor,avgFotos,avgVolumeProduto,MinimoVolumeProduto,MaximoVolumeProduto,pct_cama_mesa_banho,pct_beleza_saude,pct_esporte_lazer,pct_informatica_acessorios,...,pct_utilidades_domesticas,pct_relogios_presentes,pct_telefonia,pct_automotivo,pct_brinquedos,pct_cool_stuff,pct_ferramentas_jardim,pct_perfumaria,pct_bebes,pct_eletronicos
0,2018-01-01,0015a82c2db000af6aaaf3ae2ecb0532,2.000000,61920.000000,61920.0,61920.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
1,2018-01-01,001cca7ae9ae17fb1caed9dfb1094831,1.750000,21437.250000,19656.0,24640.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.625,0.0,0.0,0.0
2,2018-01-01,002100f778ceb8431b7a1020ff7ab48f,1.000000,8499.200000,4096.0,9600.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
3,2018-01-01,003554e2dce176b5555353e4f3555ac8,0.000000,21000.000000,21000.0,21000.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
4,2018-01-01,004c9cd9d87a3c30c522c48c4fc07416,0.911111,21858.511111,4176.0,136958.0,0.911111,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0


In [19]:
pathVendedorProduto = '../data/fs_vendedor_produto.csv'

In [20]:
tb_summary.to_csv(pathVendedorProduto, index = False, encoding = 'utf-8')